In [1]:
import math, pydot, graphviz
import numpy as np
import pandas as pd
from collections import defaultdict
# Split the data into train and test sets
# importing relevant libraries
from sklearn.model_selection import train_test_split

In [2]:
#!pip install pydot

In [3]:
#!pip install graphviz

In [2]:
df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

In [4]:
df = pd.read_csv(r"C:\Users\yotam\Desktop\naya\Preprocessed_data.csv")

In [15]:
df.head()

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country,isbn_num
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa,0
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada,1
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada,1
3,3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN,1
4,4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada,1


In [5]:
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])

In [6]:
#split to train andtest while keeping each user in the train and test data
train_users, test_users = train_test_split(df, test_size = 0.2 , random_state= 42)

In [7]:
x_train = train_users.drop(columns=['rating'])
y_train = train_users["rating"]

x_test = test_users.drop(columns=['rating'])
y_test = test_users["rating"]

## Implemation of TenserFlow

In [8]:
all_users = train_users["user_id"].unique()
all_books = train_users["isbn_num"].unique()

In [9]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

In [9]:
# user pipeline
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users)+1, output_dim=32)(user_as_integer)

In [10]:
# book pipeline
book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books)+1, output_dim=32)(book_as_integer)

In [11]:
# dot product
dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
flatten = tf.keras.layers.Flatten()(dot)

In [12]:
# model input/output definition
model = tf.keras.Model(inputs=[user_input, book_input], outputs=flatten)

In [13]:
model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [14]:
## this model is quite bad with an avg of 3 starts in rating deviation
# model.fit(
#     x={
#         "user": x_train["user_id"],
#         "book": x_train["isbn_num"]
#     },
#     y=y_train.values,
#     batch_size=256,
#     epochs=100,
#     validation_split=0.1, # for early stopping
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)
#     ],
# )


Epoch 1/100
2538/2538 [==============================] - 57s 22ms/step - loss: 22.8869 - mean_absolute_error: 2.8379 - val_loss: 23.1210 - val_mean_absolute_error: 2.8641
Epoch 2/100
2538/2538 [==============================] - 59s 23ms/step - loss: 22.8537 - mean_absolute_error: 2.8359 - val_loss: 23.1175 - val_mean_absolute_error: 2.8655
Epoch 3/100
  14/2538 [..............................] - ETA: 1:26 - loss: 22.9367 - mean_absolute_error: 2.8693

KeyboardInterrupt: 

# full pipline

In [10]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)

dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
squash = tf.keras.layers.Lambda(lambda x: 4 * tf.nn.sigmoid(x) + 1)(flatten)

model = tf.keras.Model(inputs=[user_input, book_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError()])


In [20]:
 model.fit(
     x={
         "user": x_train["user_id"],
         "book": x_train["isbn_num"]
     },
     y=y_train.values,
     batch_size=256,
     epochs=4,
     validation_split=0.1, # for early stopping
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)
     ],
 )


Epoch 1/4
2098/2901 [====================>.........] - ETA: 59s - loss: 14.3701 - mean_absolute_error: 3.5245

KeyboardInterrupt: 

add user data

In [27]:
user_input = tf.keras.layers.Input(shape=(2,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)

dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
squash = tf.keras.layers.Lambda(lambda x: 4 * tf.nn.sigmoid(x) + 1)(flatten)

model = tf.keras.Model(inputs=[user_input, book_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [28]:
 model.fit(
     x={
         "user": x_train[["user_id",'age']],
         "book": x_train["isbn_num"]
     },
     y=y_train.values,
     batch_size=256,
     epochs=5,
     validation_split=0.1, # for early stopping
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
     ],
 )


Epoch 1/5
2901/2901 [==============================] - 153s 53ms/step - loss: 14.5075 - mean_absolute_error: 3.5297 - root_mean_squared_error: 3.8089 - val_loss: 14.3180 - val_mean_absolute_error: 3.4756 - val_root_mean_squared_error: 3.7839
Epoch 2/5
2901/2901 [==============================] - 150s 52ms/step - loss: 13.9449 - mean_absolute_error: 3.3842 - root_mean_squared_error: 3.7343 - val_loss: 14.1711 - val_mean_absolute_error: 3.3803 - val_root_mean_squared_error: 3.7644
Epoch 3/5
2901/2901 [==============================] - 156s 54ms/step - loss: 13.5081 - mean_absolute_error: 3.2392 - root_mean_squared_error: 3.6753 - val_loss: 14.2842 - val_mean_absolute_error: 3.3059 - val_root_mean_squared_error: 3.7794
Epoch 4/5
2901/2901 [==============================] - 158s 54ms/step - loss: 13.0820 - mean_absolute_error: 3.1004 - root_mean_squared_error: 3.6169 - val_loss: 14.4152 - val_mean_absolute_error: 3.2619 - val_root_mean_squared_error: 3.7967


In [14]:
pred = model.predict(x_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [21]:
pred = model.predict({
    "user": x_test[["user_id",'age']],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"]#.reshape(-1, 1)
})


6445/6445 [==============================] - 7s 1ms/step


In [29]:
model.evaluate({
    "user": x_test[["user_id",'age']],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"]#.reshape(-1, 1)
},y_test)

6445/6445 [==============================] - 9s 1ms/step - loss: 14.0976 - mean_absolute_error: 3.3696 - root_mean_squared_error: 3.7547


[14.097579002380371, 3.3696367740631104, 3.7546746730804443]

In [23]:
y_test

392141    0
120888    0
472434    0
206129    5
553117    0
         ..
580066    0
861273    0
666398    0
859941    0
667974    7
Name: rating, Length: 206235, dtype: int64

In [22]:
pred

array([[3.0711696, 3.445534 ],
       [2.91386  , 4.895115 ],
       [2.9571815, 3.5963862],
       ...,
       [2.9984808, 2.8038635],
       [3.0161195, 3.0132391],
       [2.9818997, 2.8458679]], dtype=float32)

In [9]:
import keras
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils import plot_model
from keras.utils import model_to_dot
keras.utils.pydot = pydot
plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
